#  CIP Data Collection, Integration and Preprocessing
### Fall Semester 2023  
<br>

## Project

The proposed goal for this assessment is data collection through scraping. Our group wishes to provide Switzerland's tourists and infrequent skiers that may consider visiting one of the multiple swiss resorts to do ski, an overview of the resorts available, their infrastructure, conditions, but also information on accomodation and rental availability and costs.    
<br>
<br>


### Individual task  

**Resort information collection** .  

The present notebook focuses on the work performed to obtain the information regarding the resorts. For that, we collected information from https://snow.myswitzerland.com/ .  
Here we can find detailed information about each Ski Resort in Switzerland, such as infrastructure available, depth of snow, temperature, prices split per age group, ... 

Among all "snow" facilities, there is a total of 183 resorts where it is possible to ski and it is for those 183 that the information here is being collected.
<br>
<br>



### Notebook Structure:

[Library Import](#Library-Import)

[Variable Assignment](#Variable-Assignment)

[Initial Page Collection](#Initial-Page-Collection)

[Individual Resort Data Collection](#Individual-Resort-Data-Collection)

#### Library Import

To start the project, it is necessary to import the necessary Python libraries. 

In [8]:
import requests  
from bs4 import BeautifulSoup 
import re   

import pandas as pd   
import numpy as np    

from datetime import datetime, timedelta
import time

import warnings   
warnings.filterwarnings('ignore')   # Suppressing warnings to improve code readability

#### Variable Assignment 

In this section, the variables that hold the URLs to be used throughout the process of data collection are created.
There are multiple pages with the content that we want to collect, so <b>page_url</b> is used as the basis path for the HTTP requests. Later, a for loop will complete the URL with the missing information (page number).
The variable <b>base_url</b> will be used later as the basis for the individual requests per resort.

In [7]:
page_url = "https://snow.myswitzerland.com/snow_reports/piste/?p="   # incomplete after "[...]/?p="   -> the structure of the URL contains an integer after "=" to indicate the page 
base_url = "https://snow.myswitzerland.com/"

<br>

#### Initial Page Collection

The code below is responsible for scraping data from multiple pages of the website (a total of 7), specifically extracting information from table rows with a particular class. <br>
Each page displays the resorts in a list format and a preview of their information (such as resort name, location (City and Canton), the condition of the slopes and the number of lifts available).   
After inspecting the page, we know that the items in the list (30 per page) are contained in the class **"FilterGridTable--row"**, which is the information that will be provided to the "find_all" function of BeautifulSoup to find all instances of this class and save the result in a list called **all_resorts_html**.  

The data from each page is appended to the **all_data_from_url** list, which will contain the HTML that belongs to each page. From this list, the information is later separated based on its category and added to the dataframe with the relevant details per resort in each row.

In [9]:
all_data_from_url = []    # The data saved in this list will be later used again for extracting the relevant data
list_of_links = []


for page_number in range(1, 8):     # The range is set from 1 (number of the page where it starts) to 8 (since 8 is not included, the loop will iterate through pages numbered 1 to 7)
    url = page_url + str(page_number)     # Building the URL by combining the page_url and the integer that corresponds to the page number. This is updated in each iteration of this for loop
    response = requests.get(url)    # requests.get() sends an HTTP request to the URL built previously and stores the response in the variable "response"
    print("Status code of the request:", response.status_code)   
    html_content = response.content     # Extracting the content of the response (in HTML format -> content of the webpage)
    soup = BeautifulSoup(html_content, 'html.parser')     # Creating a BeautifulSoup object to parse the HTML content and stors it in the variable "soup"

    all_resorts_html = soup.find_all('tr', {'class': 'FilterGridTable--row'})   # Iterates through the entire soup object and locates all instances of the desired HTML portion per page
    print(f"Number of resorts listed in page {page_number}: ", len(all_resorts_html))

    for row in all_resorts_html:    # Extracting and storing the data from the current page by appending each row to the 'all_data_from_url' list
        all_data_from_url.append(row)   
        resort_url = soup.find_all('a', {'class': 'FilterGridTable--link'}, href=True)    # Extracting and appending the URL's per resort (each row is a different resort) to the "list_of_links"
    
    # For each "FilterGridTable--link" we are only interested in its "href" attribute to generate the full URL that leads to the information on each resort
    for link in resort_url:
        href = link['href']
        full_link = base_url + href    # The "href" obtained is combined with the base_url specified earlier. 
        # This generates an impurity in the data, because the base URL ends in "/" and "href" starts with "/". The resulting URL is not valid and needs to be dealt with. 
        if full_link not in list_of_links:   # If clause used to make sure that the data is not yet in the "list_of_links".
            list_of_links.append(full_link)
    

Status code of the request: 200
Number of resorts listed in page 1:  30
Status code of the request: 200
Number of resorts listed in page 2:  30
Status code of the request: 200
Number of resorts listed in page 3:  30
Status code of the request: 200
Number of resorts listed in page 4:  30
Status code of the request: 200
Number of resorts listed in page 5:  30
Status code of the request: 200
Number of resorts listed in page 6:  30
Status code of the request: 200
Number of resorts listed in page 7:  3


In [10]:
# Inspecting the length of the all_data_from_url list to make sure the information was extracted from all known resorts.
len(all_data_from_url)

183

In [11]:
# Inspecting the length of the list of links to ensure that it is the same as the list of the resorts.
len(list_of_links)

183

After inspecting the length of both lists, we see that both have the same number of elements, <b> 183 </b>, which corresponds to the number of resorts where according to https://snow.myswitzerland.com/snow_reports/ is the number of resorts where it is possible to ski.


Having all data in a list,  we can now look for the relevant information and save it in a dataframe.
The collection of resort names is done using a for loop that for each row in the list we created before, finds all "div" elements with class "FilterGridTable--title" in the current row, extracts the text content and stores it in the "data" list.  

In the collection of the locations, list comprehension is used instead. 

In [12]:
resort_title = []

# Iterating through each row in the previously collected data
for row in all_data_from_url:
    headers = row.find_all('div', {"class":"FilterGridTable--title"})     # Finding all "div" elements with class "FilterGridTable--title" in the current row
    data = [header.get_text(strip=True) for header in headers]    # Extract the text content of the "div" elements and store them in the "data" list
    resort_title.append(data)     # Appending the extracted data (resort names) to the resort_title list

# Creating a DataFrame adding the elements the resort_title list into a column named "Resort"
resort_df = pd.DataFrame(resort_title, columns = ["Resort"])


In [13]:
data_locations = [[header.get_text(strip=True) for header in row.find_all('div', {"class":"FilterGridTable--info"})] for row in all_data_from_url]  
# The inner list comprehension ( [header.get_text(strip=True) for header in row.find_all('div', {"class":"FilterGridTable--info"})] ) extracts the text content of all 'div' elements with class "FilterGridTable--info" in each row and using it to create the data_locations list
# The outer list comprehension ( [... for row in all_data_from_url] )  is used to apply the inner list comprehension to each row of all_data_from_url. This creates a list of lists.

resort_df["Location"] = data_locations

In [14]:
resort_df.head()

,Resort,Location
0,Samnaun/Ischgl,[Samnaun – Graubünden]
1,Engadin & St. Moritz,[St. Moritz – Graubünden]
2,Verbier,[Verbier – Valais]
3,Thyon - 4 Vallées,[Les Collons Thyon – Valais]
4,St. Moritz,[St. Moritz – Graubünden]


In [ ]:
# Using a lambda function to extract elements of list
resort_df = resort_df.applymap(lambda x: x[2 : -2] if isinstance(x, str) and x.startswith("[") and x.endswith("]") else x)

In [15]:
resort_df.head()

,Resort,Location
0,Samnaun/Ischgl,[Samnaun – Graubünden]
1,Engadin & St. Moritz,[St. Moritz – Graubünden]
2,Verbier,[Verbier – Valais]
3,Thyon - 4 Vallées,[Les Collons Thyon – Valais]
4,St. Moritz,[St. Moritz – Graubünden]


Extracting "City" and "Canton" from "Location" column

In [ ]:
# Using split() to separate the elements of "Location". First part before "–" is the City and final part is the Canton
resort_df["City"] = resort_df.Location.str.split("–", expand = True)[0]
resort_df["Canton"] = resort_df.Location.str.split("–", expand = True)[1]

It is also helpful to have the URL of each resort so that we can collect the information on each resort. Instead of just scraping the main pages of the website, we also scrape the page that belongs to each resort to get additional and more complete information.

In [16]:
resort_df["Link"] = list_of_links

In [17]:
# Inspecting the URL: 
resort_df["Link"][0]

'https://snow.myswitzerland.com//snow_reports/samnaunischgl-157/#piste'

The link, as is, is not correct. The base URL ends in "/" and the "href" obtained from the soup object starts with "/". This means that the resulting URL does not follow the appropriate structure and needs to be processed further.   

This could be solved by removing the "/" from the **base_url** but this way, we get to use replace() to clean up the data. In other real world situations, the simple solution may not be available and the processing is necessary.

In [18]:
resort_df["Link"] = resort_df["Link"].str.replace('//', '/')    # Replaces all occurrences of "//" for "/"
resort_df["Link"] = resort_df["Link"].str.replace('https:/', 'https://')    # The previous replacement works for the rest of the URL, but also changes the scheme part. 

# Per the specification (https://www.rfc-editor.org/rfc/rfc3986#section-3) this is required to have "//" and so it should be processed again to ensure conformity. 
# Providing the initial part of the URL ("http:") ensures that only this section of the string is changed

# Re-inspecting the URL:
resort_df["Link"][0]

'https://snow.myswitzerland.com/snow_reports/samnaunischgl-157/#piste'

In [19]:
# Using list comprehension to create the DataFrame directly
data_content = [[content.get_text(strip=True) for content in row.find_all('td', {'class': 'FilterGridTable--cell'})] for row in all_data_from_url]

# Ensuring all lists in data_content have the same number of elements
data_content = [content + [""] * (3 - len(content)) if len(content) < 3 else content for content in data_content]


# Using NumPy to assign the data from data_content to specific columns in resort_df
resort_df[["Open Pistes km", "Slope Condition", "Open Lifts"]] = np.array(data_content)

In [20]:
resort_df.head(5)

,Resort,Location,Link,Open Pistes km,Slope Condition,Open Lifts
0,Samnaun/Ischgl,[Samnaun – Graubünden],https://snow.myswitzerland.com/snow_reports/sa...,"204.0/239.0 kmPistes, good",goodSlope condition,40/46Lifts open
1,Engadin & St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/en...,"126.5/330.3 kmPistes, good",goodSlope condition,26/59Lifts open
2,Verbier,[Verbier – Valais],https://snow.myswitzerland.com/snow_reports/ve...,"92.1/358.0 kmPistes, good",goodSlope condition,22/73Lifts open
3,Thyon - 4 Vallées,[Les Collons Thyon – Valais],https://snow.myswitzerland.com/snow_reports/th...,"92.1/358.0 kmPistes, good",goodSlope condition,22/73Lifts open
4,St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/st...,"81.0/163.8 kmPistes, good",goodSlope condition,19/24Lifts open


**Impurities** - removing part of a string

The obtained information has text that is not relevant (e.g. "204.0/239.0 kmPistes, good", "goodSlope condition", "38/46Lifts open"). strip() is used to split the string at the specified substring and the [0] index is used to select the first part of the split string.

In [21]:
resort_df['Open Pistes km'] = resort_df['Open Pistes km'].str.split('km').str[0]    # Extracting numerical information from the "Open Pistes km" column by splitting the string at 'km' and keeping the first part
resort_df['Slope Condition'] = resort_df['Slope Condition'].str.split('Slope condition').str[0]     # Extracting slope condition information by splitting the string at 'Slope condition' and keeping the first part
resort_df['Open Lifts'] = resort_df['Open Lifts'].str.split('Lifts open').str[0]     # Extracting information about number ofopen lifts by splitting the string at 'Lifts open' and keeping the first part


In [22]:
resort_df.head()

,Resort,Location,Link,Open Pistes km,Slope Condition,Open Lifts
0,Samnaun/Ischgl,[Samnaun – Graubünden],https://snow.myswitzerland.com/snow_reports/sa...,204.0/239.0,good,40/46
1,Engadin & St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/en...,126.5/330.3,good,26/59
2,Verbier,[Verbier – Valais],https://snow.myswitzerland.com/snow_reports/ve...,92.1/358.0,good,22/73
3,Thyon - 4 Vallées,[Les Collons Thyon – Valais],https://snow.myswitzerland.com/snow_reports/th...,92.1/358.0,good,22/73
4,St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/st...,81.0/163.8,good,19/24


#### Individual Resort Data Collection

After collecting all the useful information found in the global pages of the website, there's additional information that can be collected by scraping each resort's individual URL colllected earlier.

In [23]:
scraped_data = []

start_time = time.perf_counter()

for link in resort_df["Link"]:
    response = requests.get(link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")

    table_rows_resort = soup.find_all("div", {"class": "KeyValueList firstColumnStretchMore"})

    link_data = {}    # Creating a dictionary to store the scraped data for this link


    for row in table_rows_resort:
        cont_test = row.find_all("th", {"scope": "row"})
        test_test = row.find_all("td")

        # Extract the text content of table elements (<th> for the header, <td> for the data)
        th_values = [content.get_text(strip = True) for content in cont_test]
        td_values = [con.get_text(strip = True) for con in test_test]

        link_data.update(dict(zip(th_values, td_values)))     # Updating the link_data dictionary with the scraped data

    # Appending the data from the current link to the scraped_df
    scraped_data.append(link_data)

# Creating a DataFrame from the list of scraped data
scraped_df = pd.DataFrame(scraped_data)

end_time = time.perf_counter()
elapsed_time = end_time - start_time
print("Running time: ", elapsed_time)

# Concatenating the scraped data with the original DataFrame
resort_df = pd.concat([resort_df, scraped_df], axis=1)


Running time:  368.321275246999


In [24]:
scraped_df.shape

(183, 227)

In [25]:
resort_df.head()

,Resort,Location,Link,Open Pistes km,Slope Condition,Open Lifts,Depth of snow in resortat 1840 metres,Depth of snow on upper runsat 2488 metres,Last snowfall in resort,Last snowfall in ski area,...,Depth of snow on upper runsat 2552 metres,Depth of snow on upper runsat 1450 metres,Depth of snow in resortat 1420 metres,Depth of snow on upper runsat 2170 metres,Depth of snow in resortat 1750 metres,Depth of snow on upper runsat 2270 metres,Depth of snow in resortat 1230 metres,Depth of snow in resortat 740 metres,Depth of snow in resortat 930 metres,Depth of snow on upper runsat 1470 metres
0,Samnaun/Ischgl,[Samnaun – Graubünden],https://snow.myswitzerland.com/snow_reports/sa...,204.0/239.0,good,40/46,50 cm,100 cm,3 days ago,3 days ago,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Engadin & St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/en...,126.5/330.3,good,26/59,NaN,NaN,02.12.2023,02.12.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Verbier,[Verbier – Valais],https://snow.myswitzerland.com/snow_reports/ve...,92.1/358.0,good,22/73,NaN,NaN,today,today,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Thyon - 4 Vallées,[Les Collons Thyon – Valais],https://snow.myswitzerland.com/snow_reports/th...,92.1/358.0,good,22/73,NaN,NaN,today,today,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/st...,81.0/163.8,good,19/24,NaN,NaN,02.12.2023,02.12.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Impurities** - Word mispelling

In the obtained information, the key that is used contains a typo created by a missing " ". replace() is used, since there are only 2 occurrences of this type of misspelling.  

In [26]:
for col in resort_df.columns:
    if "resortat" in col:     # Checking if the substring "resortat" is present in the column name
        new_col = col.replace("resortat", "resort at")   # Replace "resortat" with "resort at" in the column name
        resort_df = resort_df.rename(columns={col: new_col})     # Rename the column in the resort_df DataFrame

In [27]:
for col in resort_df.columns:
    if "runsat" in col:
        new_col = col.replace("runsat", "runs at")
        resort_df = resort_df.rename(columns={col: new_col})

In [28]:
resort_df.head()

,Resort,Location,Link,Open Pistes km,Slope Condition,Open Lifts,Depth of snow in resort at 1840 metres,Depth of snow on upper runs at 2488 metres,Last snowfall in resort,Last snowfall in ski area,...,Depth of snow on upper runs at 2552 metres,Depth of snow on upper runs at 1450 metres,Depth of snow in resort at 1420 metres,Depth of snow on upper runs at 2170 metres,Depth of snow in resort at 1750 metres,Depth of snow on upper runs at 2270 metres,Depth of snow in resort at 1230 metres,Depth of snow in resort at 740 metres,Depth of snow in resort at 930 metres,Depth of snow on upper runs at 1470 metres
0,Samnaun/Ischgl,[Samnaun – Graubünden],https://snow.myswitzerland.com/snow_reports/sa...,204.0/239.0,good,40/46,50 cm,100 cm,3 days ago,3 days ago,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Engadin & St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/en...,126.5/330.3,good,26/59,NaN,NaN,02.12.2023,02.12.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Verbier,[Verbier – Valais],https://snow.myswitzerland.com/snow_reports/ve...,92.1/358.0,good,22/73,NaN,NaN,today,today,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Thyon - 4 Vallées,[Les Collons Thyon – Valais],https://snow.myswitzerland.com/snow_reports/th...,92.1/358.0,good,22/73,NaN,NaN,today,today,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/st...,81.0/163.8,good,19/24,NaN,NaN,02.12.2023,02.12.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Impurities** - merging duplicated "Depth of snow" columns

Since each resort can have a different altitude, it results in 227 columns with "Depth of snow" information, but most are filled with NaN, so the columns are merged into a single one with "Depth of snow in resort" and another with the "Depth of snow on the upper runs" using the apply() function to apply a custom lambda function to each row along columns. 

The lambda function takes row x, drops any  missing values, and then joins the non-missing values into a single string separated by commas.

In [29]:
depth_cols_resort = [col for col in resort_df.columns if 'Depth of snow in resort' in col]
depth_cols_runs = [col for col in resort_df.columns if 'Depth of snow on upper runs' in col]  # higher in the mountain

In [30]:
resort_df['Depth of snow in resort'] = resort_df[depth_cols_resort].apply(lambda x: ','.join(x.dropna()), axis=1)
resort_df['Depth of snow on upper runs'] = resort_df[depth_cols_runs].apply(lambda x: ','.join(x.dropna()), axis=1)

In [31]:
resort_df.head()

,Resort,Location,Link,Open Pistes km,Slope Condition,Open Lifts,Depth of snow in resort at 1840 metres,Depth of snow on upper runs at 2488 metres,Last snowfall in resort,Last snowfall in ski area,...,Depth of snow on upper runs at 2552 metres,Depth of snow on upper runs at 1450 metres,Depth of snow in resort at 1420 metres,Depth of snow on upper runs at 2170 metres,Depth of snow in resort at 1750 metres,Depth of snow on upper runs at 2270 metres,Depth of snow in resort at 1230 metres,Depth of snow in resort at 740 metres,Depth of snow in resort at 930 metres,Depth of snow on upper runs at 1470 metres
0,Samnaun/Ischgl,[Samnaun – Graubünden],https://snow.myswitzerland.com/snow_reports/sa...,204.0/239.0,good,40/46,50 cm,100 cm,3 days ago,3 days ago,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Engadin & St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/en...,126.5/330.3,good,26/59,NaN,NaN,02.12.2023,02.12.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Verbier,[Verbier – Valais],https://snow.myswitzerland.com/snow_reports/ve...,92.1/358.0,good,22/73,NaN,NaN,today,today,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Thyon - 4 Vallées,[Les Collons Thyon – Valais],https://snow.myswitzerland.com/snow_reports/th...,92.1/358.0,good,22/73,NaN,NaN,today,today,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/st...,81.0/163.8,good,19/24,NaN,NaN,02.12.2023,02.12.2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Dropping all the columns with the word "metres" (our merged columns do not contain it)
resort_df.drop(list(resort_df.filter(regex='metres')), axis=1, inplace=True)

In [33]:
resort_df.shape

(183, 39)

**Impurities** - accurate time conversion

In columns such as "Last snowfall in resort", some of the rows contain a specific date, but others say something like "today", "2 days ago". Here those cases are processed to get the specific date.

In [34]:
# Getting today's date
today = datetime.now()

def extract_days(s):
  match = re.search(r'\d+', s)
  return int(match.group()) if match else 0

# Apply function to 'Last snowfall in resort' column
resort_df['Last snowfall in resort'] = resort_df['Last snowfall in resort'].apply(lambda x: (today - timedelta(days=extract_days(str(x)))).date())
resort_df['Last snowfall in ski area'] = resort_df['Last snowfall in ski area'].apply(lambda x: (today - timedelta(days=extract_days(str(x)))).date())


**Impurities** - metric system conversion  
Non-Swiss skiers could come from countries that use the Imperial System instead of the Metric System. TO adapt the data for those cases, additional colums are added to the dataframe and the conversion is done using the function cm_to_inches().

In [35]:
def cm_to_inches(cm_value):
    try:
        if cm_value.strip() == '-' or cm_value.strip() == '':
            return 'Not available'
        return round(float(cm_value.split()[0]) / 2.54, 2)
    except (ValueError, IndexError):
        return None

def handle_nan_inches(value):
    if pd.isna(value):
        return 'Not available'
    return value

# Convert and rename 'Depth of snow on upper runs' column
resort_df['Depth of snow on upper runs - inches'] = resort_df['Depth of snow on upper runs'].apply(cm_to_inches)
resort_df['Depth of snow on upper runs - inches'] = resort_df['Depth of snow on upper runs - inches'].apply(handle_nan_inches)
resort_df.rename(columns={'Depth of snow on upper runs': 'Depth of snow on upper runs - cm'}, inplace=True)

# Convert and rename 'Depth of snow in resort' column
resort_df['Depth of snow in resort - inches'] = resort_df['Depth of snow in resort'].apply(cm_to_inches)
resort_df['Depth of snow in resort - inches'] = resort_df['Depth of snow in resort - inches'].apply(handle_nan_inches)
resort_df.rename(columns={'Depth of snow in resort': 'Depth of snow in resort - cm'}, inplace=True)

In [ ]:
resort_df["New snow in resort (past 24h)"] = resort_df["New snow in resort (past 24h)"].str.split("cm").str[0]    
resort_df["New snow in ski area (past 24h)"] = resort_df["New snow in ski area (past 24h)"].str.split("cm").str[0]    
resort_df["Surface area of pistes"] = resort_df["Surface area of pistes"].str.split("km").str[0]    
resort_df["Pistes covered by lift pass (regional)"] = resort_df["Pistes covered by lift pass (regional)"].str.split("km").str[0]    

resort_df.rename(columns={"Surface area of pistes": "Surface area of pistes km^2"}, inplace=True)
resort_df.rename(columns={"Pistes covered by lift pass (regional)": "Pistes covered by lift pass (regional) - km"}, inplace=True)
resort_df.rename(columns={"New snow in resort (past 24h)": "New snow in resort (past 24h) - cm"}, inplace=True)
resort_df.rename(columns={"New snow in ski area (past 24h)": "New snow in ski area (past 24h) - cm"}, inplace=True)


In [37]:
hours_data = []
start_time = time.perf_counter()

for link in resort_df["Link"]:
    response = requests.get(link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")

    resort_op_hours = soup.find_all("div", {"class": "ArticleSection", "id": "articlesection-u20"})

    link_data_hours = {}

    for row in resort_op_hours:
        cont_test = row.find_all("th", {"scope": "row"})
        test_test = row.find_all("td")

        th_values = [content.get_text(strip = True) for content in cont_test]
        td_values = [con.get_text(strip = True) for con in test_test]

        link_data_hours.update(dict(zip(th_values, td_values)))

    hours_data.append(link_data_hours)

opening_hours = pd.DataFrame(hours_data)

end_time = time.perf_counter()
elapsed_time = end_time - start_time
print("Running time: ", elapsed_time)

resort_df = pd.concat([resort_df, opening_hours], axis=1)

Running time:  365.1225871030001


In [38]:
opening_hours.head()

,Installations open from,Installations open until,Start of season:,End of season:
0,08:30 hr,16:00 hr,23.11.2023,01.05.2024
1,NaN,NaN,21.10.2023,05.05.2024
2,08:45 hr,16:15 hr,04.11.2023,21.04.2024
3,08:30 hr,16:30 hr,17.11.2023,14.04.2024
4,07:45 hr,17:00 hr,25.11.2023,07.04.2024


In [39]:
prices_data = []
start_time = time.perf_counter()

for link in resort_df["Link"]:
    response = requests.get(link)
    html_content = response.content
    soup = BeautifulSoup(html_content, "html.parser")

    resort_prices = soup.find_all("div", {"class": "ArticleSubSection--content"})

    link_data_prices = {}

    for row in resort_prices:
        cont_test = row.find_all("th", {"scope": "row"})
        test_test = row.find_all("td")
        

        th_values = [content.get_text(strip=True) for content in cont_test]
        td_values = [con.get_text(strip=True) for con in test_test]
        
        # Some links have a section of text before the prices with the same HTML tag and without a table, it returns an empty list.
        # This if clause ensures that if the list is empty due to that initial section, it runs again until the table was filled.
        if len(th_values) == 0:    
            continue      
        else:
            break
    
    # Checking if the key already exists in link_data_prices (there is more than one table and the categories are the same)
    # In case we find multiple values, for instance, for "Adult", instead of getting the last value, we get 2 keys, "Adult" and "Adult_1"
    for key, value in zip(th_values, td_values):
        
        counter = 1
        new_key = key
        while new_key in link_data_prices:
            # If key exists, it appends a counter to make it unique
            counter += 1
            new_key = f"{key}_{counter}"

        link_data_prices[new_key] = value

    prices_data.append(link_data_prices)

resort_prices = pd.DataFrame(prices_data)

end_time = time.perf_counter()
elapsed_time = end_time - start_time
print("Running time: ", elapsed_time)

resort_df = pd.concat([resort_df, resort_prices], axis=1)

Running time:  369.14527432100294


In [40]:
resort_df.shape

(183, 148)

**Impurities** - merging duplicated columns

As mentioned, the names of the categories can vary from one resort to another. Here, all columns with a specific string are saved into a variable wich is then used to create new columns in the dataframe and all values are merged into the new columns. 

In [41]:
# Saving all the columns that contain the specified substring
adult_columns = resort_df.filter(like="Adult").columns
young_columns = resort_df.filter(like="Young").columns
senior_columns = resort_df.filter(like="Senior").columns
child_columns = resort_df.filter(like="Child").columns

# Merging the columns into a single one
resort_df["adult - CHF"] = resort_df[adult_columns].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
resort_df["young person - CHF"] = resort_df[young_columns].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
resort_df["senior - CHF"] = resort_df[senior_columns].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
resort_df["children - CHF"] = resort_df[child_columns].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

In [42]:
resort_df.head()

,Resort,Location,Link,Open Pistes km,Slope Condition,Open Lifts,Last snowfall in resort,Last snowfall in ski area,New snow in resort (past 24h),New snow in ski area (past 24h),...,Young person: 17-20,Children: 9-16,Young person: 20,Children: 20,Young person: 14-17,Children: 16,adult - CHF,young person - CHF,senior - CHF,children - CHF
0,Samnaun/Ischgl,[Samnaun – Graubünden],https://snow.myswitzerland.com/snow_reports/sa...,204.0/239.0,good,40/46,2023-12-05,2023-12-05,0 cm,35 cm,...,NaN,NaN,NaN,NaN,NaN,NaN,72.- CHF,–,72.- CHF,45.- CHF
1,Engadin & St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/en...,126.5/330.3,good,26/59,2023-12-06,2023-12-06,0 cm,0 cm,...,NaN,NaN,NaN,NaN,NaN,NaN,from 45.- CHF,from 29.50 CHF,–,from 15.- CHF
2,Verbier,[Verbier – Valais],https://snow.myswitzerland.com/snow_reports/ve...,92.1/358.0,good,22/73,2023-12-08,2023-12-08,2 cm,2 cm,...,NaN,NaN,NaN,NaN,NaN,NaN,from 83.- CHF,from 70.- CHF,–,from 41.- CHF
3,Thyon - 4 Vallées,[Les Collons Thyon – Valais],https://snow.myswitzerland.com/snow_reports/th...,92.1/358.0,good,22/73,2023-12-08,2023-12-08,0 cm,0 cm,...,NaN,NaN,NaN,NaN,NaN,NaN,73.- CHF,62.- CHF,–,37.- CHF
4,St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/st...,81.0/163.8,good,19/24,2023-12-06,2023-12-06,0 cm,0 cm,...,NaN,NaN,NaN,NaN,NaN,NaN,from 45.- CHF,from 29.50 CHF,–,from 15.- CHF


**Impurities** - filling missing values

Some resorts don't have a specific price difference between Adult and Senior individuals. For that reason, the missing values in the "senior - CHF" column are populated by that row's "adult - CHF" values, to avoid having missing values. The same thing is done to the "young person - CHF" column, since there's no distinction in the prices. 

In [43]:
resort_df["senior - CHF"] = resort_df["senior - CHF"].fillna(resort_df["adult - CHF"])
resort_df["young person - CHF"] = resort_df["young person - CHF"].fillna(resort_df["adult - CHF"])


In [44]:
# After processing, all unnecessary price columns can now be dropped

resort_df.drop(list(resort_df.filter(regex='Adult')), axis=1, inplace=True)
resort_df.drop(list(resort_df.filter(regex='Young')), axis=1, inplace=True)
resort_df.drop(list(resort_df.filter(regex='Senior')), axis=1, inplace=True)
resort_df.drop(list(resort_df.filter(regex='Child')), axis=1, inplace=True)

In [45]:
resort_df.head()

,Resort,Location,Link,Open Pistes km,Slope Condition,Open Lifts,Last snowfall in resort,Last snowfall in ski area,New snow in resort (past 24h),New snow in ski area (past 24h),...,Depth of snow on upper runs - inches,Depth of snow in resort - inches,Installations open from,Installations open until,Start of season:,End of season:,adult - CHF,young person - CHF,senior - CHF,children - CHF
0,Samnaun/Ischgl,[Samnaun – Graubünden],https://snow.myswitzerland.com/snow_reports/sa...,204.0/239.0,good,40/46,2023-12-05,2023-12-05,0 cm,35 cm,...,39.37,19.69,08:30 hr,16:00 hr,23.11.2023,01.05.2024,72.- CHF,–,72.- CHF,45.- CHF
1,Engadin & St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/en...,126.5/330.3,good,26/59,2023-12-06,2023-12-06,0 cm,0 cm,...,53.94,21.65,NaN,NaN,21.10.2023,05.05.2024,from 45.- CHF,from 29.50 CHF,–,from 15.- CHF
2,Verbier,[Verbier – Valais],https://snow.myswitzerland.com/snow_reports/ve...,92.1/358.0,good,22/73,2023-12-08,2023-12-08,2 cm,2 cm,...,35.83,19.69,08:45 hr,16:15 hr,04.11.2023,21.04.2024,from 83.- CHF,from 70.- CHF,–,from 41.- CHF
3,Thyon - 4 Vallées,[Les Collons Thyon – Valais],https://snow.myswitzerland.com/snow_reports/th...,92.1/358.0,good,22/73,2023-12-08,2023-12-08,0 cm,0 cm,...,39.37,23.62,08:30 hr,16:30 hr,17.11.2023,14.04.2024,73.- CHF,62.- CHF,–,37.- CHF
4,St. Moritz,[St. Moritz – Graubünden],https://snow.myswitzerland.com/snow_reports/st...,81.0/163.8,good,19/24,2023-12-06,2023-12-06,0 cm,0 cm,...,35.43,19.69,07:45 hr,17:00 hr,25.11.2023,07.04.2024,from 45.- CHF,from 29.50 CHF,–,from 15.- CHF


Inspecting the dataframe to know if there are additional columns that can be dropped or that need further processing. 

Using both isna() and isnull() to check for both NaN values and empty rows.

In [46]:
# Inspecting the columns
nan_count_per_column = resort_df.isna().sum()
print(nan_count_per_column)

Resort                                                   0
Location                                                 0
Link                                                     0
Open Pistes km                                           0
Slope Condition                                          0
Open Lifts                                               0
Last snowfall in resort                                  0
Last snowfall in ski area                                0
New snow in resort (past 24h)                           11
New snow in ski area (past 24h)                         11
Runs down to resort (open today / total)                57
Runs down to (altitude)                                 44
Condition of runs down to resort                        44
Surface area of pistes                                  11
Number of local pistes                                  11
Pistes covered by lift pass (regional)                  11
Number of lifts in ski area                             

In [47]:
# Defining a threshold for the amount of missing numbers.
threshold = 70

# Droping all the columns with more missing values than the defined threshold
resort_df = resort_df.dropna(axis=1, thresh=resort_df.shape[0] - threshold)

In [48]:
resort_df.shape

(183, 34)

In [49]:
for column in resort_df.columns:
    if resort_df[column].isna().sum() <= threshold:
        resort_df[column].fillna("Not Available", inplace=True)

# Print or use the modified DataFrame
resort_df.shape

(183, 34)

In [50]:
# Inspecting the columns
nan_count_per_column = resort_df.isna().sum()
print(nan_count_per_column)

Resort                                      0
Location                                    0
Link                                        0
Open Pistes km                              0
Slope Condition                             0
Open Lifts                                  0
Last snowfall in resort                     0
Last snowfall in ski area                   0
New snow in resort (past 24h)               0
New snow in ski area (past 24h)             0
Runs down to resort (open today / total)    0
Runs down to (altitude)                     0
Condition of runs down to resort            0
Surface area of pistes                      0
Number of local pistes                      0
Pistes covered by lift pass (regional)      0
Number of lifts in ski area                 0
Runs floodlit today until                   0
Length of floodlit runs                     0
Winter walking paths from (altitude)        0
Number of snow shoe trails open today       0
Conditions/events on the trails   

In [51]:
resort_df.shape

(183, 34)

In [52]:
resort_df.to_csv("../Data/resorts_output.csv")